In [9]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

In [1]:
def count(motifs):
    count = {}
    k = len(motifs[0])
    for symbol in 'ACGT':
        count[symbol] = []
        for i in range(k):
            count[symbol].append(0)
    t = len(motifs)
    for i in range(t):
        for j in range(k):
            symbol = motifs[i][j]
            count[symbol][j] += 1 
    return count

{'A': [1, 2, 1, 0, 0, 2], 'C': [2, 1, 4, 2, 0, 0], 'G': [1, 1, 0, 2, 1, 1], 'T': [1, 1, 0, 1, 4, 2]}


In [5]:
def profile(motifs):
    profile = {}
    c = count(motifs)
    for key in c:
        profile[key] = []
        for i in range(len(c[key])):
            profile[key].append(c[key][i]/10)
    return profile


{'A': [0.1, 0.2, 0.1, 0.0, 0.0, 0.2], 'C': [0.2, 0.1, 0.4, 0.2, 0.0, 0.0], 'G': [0.1, 0.1, 0.0, 0.2, 0.1, 0.1], 'T': [0.1, 0.1, 0.0, 0.1, 0.4, 0.2]}


In [7]:
def consensus(motifs):
    k = len(motifs[0])
    c = count(motifs)
    consensus = ""
    for j in range(k):
        selected_symbol = ""
        max_count = 0
        for symbol in "ACGT":
            if c[symbol][j] > max_count:
                max_count = c[symbol][j]
                selected_symbol = symbol
        consensus += selected_symbol
    return consensus

CACCTA


In [8]:
def score(motifs):
    k = len(motifs[0])
    t = len(motifs)
    c = consensus(motifs)
    s = 0
    for i in range(t):
        for j in range(k):
            if motifs[i][j] != c[j]:
                s += 1
    return s

input = ["AACGTA","CCCGTT", "CACCTT","GGATTA","TTCCGG"]
print(score(input))

14


In [24]:
def entropy(motifs):
    k = len(motifs[0])
    p = profile(motifs)
    e = 0
    for i in range(k):
        score = 0
        for symbol in "ACGT":
            if p[symbol][i] != 0 and p[symbol][i] != 1:
                score += -(p[symbol][i] * math.log(p[symbol][i], 2))
        e += score   
    return e

input = ["AACGTA","CCCGTT", "CACCTT","GGATTA","TTCCGG"]
print(entropy(input))

7.165784284662087


In [ ]:
def pr(pattern, profile):
    k = len(pattern)
    product = 1
    for i in range(k):
        product *= profile[pattern[i]][i]
    return product

def profile_most_probable_kmer(text, k, profile):
    score_highest = -1
    pattern_highest = ""
    for i in range(len(text) - k + 1):
        score = pr(text[i:i+k], profile)
        if score > score_highest:
            score_highest = score
            pattern_highest = text[i:i+k]
    return pattern_highest


In [ ]:
def greedy_motif_search(dna, k, t):
    best_motifs = []
    for i in range(0, t):
        best_motifs.append(dna[i][0:k])
    n = len(dna[0])
    for i in range(n-k+1):
        motifs = []
        motifs.append(dna[0][i:i+k])
        for j in range(1, t):
            p = profile(motifs[0:j])
            motifs.append(profile_most_probable_kmer(dna[j], k, p))
        if score(motifs) < score(best_motifs):
            best_motifs = motifs
    return best_motifs